In [2]:
import numpy as np
import pandas as pd
import pickle
#path stuff
from pathlib import WindowsPath, PosixPath
import sys
PathPath = WindowsPath if 'win' in sys.platform.lower() else PosixPath
PARENT_DIR = PathPath('../')
INFERENCE_PATH = PARENT_DIR.joinpath('.inference')  
GIT_TABLE_PATH = PARENT_DIR.joinpath('.git_tables')
sys.path.append(str(PARENT_DIR))
from common_util import _FILE_TO_LABEL as file_to_label,markdown_table_from_df,hdi

### Extract Model Inadequcies for Comparison
compare the calibrated model inadquency between the combined inference and independent inference to evaluate the impact of considering each of the datasets together.

In [4]:
model_name = 'JMAK_{}'

data_names = [v for k,v in file_to_label.items() if 'shah' not in k.lower()]

sigma = []
for data_name in data_names:
    with open(INFERENCE_PATH.joinpath(f'{model_name.format(data_name)}_trunc_normal_samples.pkl'),'rb') as f:
        samples = pd.DataFrame.from_dict(pickle.load(f))
    sigma.append(samples['sigma'].mean())

individual_sigma = pd.Series(sigma,index=data_names,
                             name = 'individual')

with open(INFERENCE_PATH.joinpath(f'{model_name.format("hierarchical")}_trunc_normal_samples.pkl'),'rb') as f:
    posterior_samples = pickle.load(f)

combined_sigma = pd.Series(posterior_samples['sigma'].mean(axis=0),index=data_names,
                           name = 'combined')

model_error = pd.concat([individual_sigma,combined_sigma],axis=1)
table = markdown_table_from_df(np.round(model_error,4),
                               'Independent vs. Combined Error Comparison',
                               'Recrystallization model inadequecy standard deviation comparison across datasets')

with open(GIT_TABLE_PATH.joinpath('model_error_comparison_truncated_normal.tbl'),'w') as f:    
    f.write(table)


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

### Put Estimated Parameters into a table

mean, ML and upper/lower bounds. 

In [ ]:
def vr(x_: float) -> float:
    x = abs(x_)
    if x >= 1:
        return round(x_,1)
    elif x < 1 and x > 0.1:
        return round(x_,2)
    elif x <= 0.1 and x > 0.01:
        return round(x_,3)
    else:
        return 0
    

def format_cell(mean: float,
                ml: float,
                std: float) -> str:
    
    return f'${vr(mean)} \pm {vr(std)} (\mathbf'+ '{' + str(vr(ml)) + '})'

def format_row(summary: pd.DataFrame) -> str:
    
    text = ''
    for param in summary.columns:
        text += format_cell(summary.loc['mean',param],
                            summary.loc['ml',param],
                            summary.loc['std',param])
        text += ' & '
    
    return text[:-3]

model_name = 'Generalized Logistic_{}'

data_names = [v for k,v in file_to_label.items()]

text = ''
for data_name in data_names:
    inf_summary = pd.read_csv(INFERENCE_PATH.joinpath(f'{model_name.format(data_name)}_trunc_normal_params.csv'),index_col = 0).T
    inf_summary['B1']/=1e4
    inf_summary['B2']/=1e4
    row = data_name + ' & ' + format_row(round(inf_summary,2)) + '\n'
    text += row

print(text)





Lopez et al. (2015) - HR & $13.2 \pm 1.8 (\mathbf{14.2}) & $-3.4 \pm 0.25 (\mathbf{-3.6}) & $-21.0 \pm 1.6 (\mathbf{-20.0}) & $4.6 \pm 0.21 (\mathbf{4.5}) & $1.7 \pm 0.38 (\mathbf{1.5}) & $0.03 \pm 0 (\mathbf{0})
Lopez et al. (2015) - MR & $33.3 \pm 0.52 (\mathbf{33.8}) & $-7.0 \pm 0.08 (\mathbf{-7.0}) & $-45.2 \pm 18.2 (\mathbf{-8.6}) & $4.8 \pm 2.7 (\mathbf{2.5}) & $3.4 \pm 0.39 (\mathbf{3.2}) & $0.08 \pm 0 (\mathbf{0.06})
Richou et al. (2020) - Batch A & $27.4 \pm 1.4 (\mathbf{27.0}) & $-6.2 \pm 0.25 (\mathbf{-6.1}) & $-45.0 \pm 44.6 (\mathbf{-26.7}) & $-2.0 \pm 7.9 (\mathbf{-7.8}) & $2.6 \pm 0.45 (\mathbf{2.5}) & $0.02 \pm 0 (\mathbf{0})
Richou et al. (2020) - Batch B & $19.7 \pm 0.99 (\mathbf{19.8}) & $-4.9 \pm 0.19 (\mathbf{-4.9}) & $-69.1 \pm 54.3 (\mathbf{-13.6}) & $-0.28 \pm 10.4 (\mathbf{-25.4}) & $3.0 \pm 0.83 (\mathbf{3.0}) & $0.02 \pm 0.02 (\mathbf{0})
Yu et al. (2017) & $5.7 \pm 2.0 (\mathbf{3.1}) & $-2.3 \pm 0.31 (\mathbf{-1.9}) & $-49.4 \pm 45.8 (\mathbf{-29.6}) & $-0.8

<>:17: SyntaxWarning: invalid escape sequence '\p'
<>:17: SyntaxWarning: invalid escape sequence '\m'
<>:17: SyntaxWarning: invalid escape sequence '\p'
<>:17: SyntaxWarning: invalid escape sequence '\m'
C:\Users\micha\AppData\Local\Temp\ipykernel_7164\2003442687.py:17: SyntaxWarning: invalid escape sequence '\p'
  return f'${vr(mean)} \pm {vr(std)} (\mathbf'+ '{' + str(vr(ml)) + '})'
C:\Users\micha\AppData\Local\Temp\ipykernel_7164\2003442687.py:17: SyntaxWarning: invalid escape sequence '\m'
  return f'${vr(mean)} \pm {vr(std)} (\mathbf'+ '{' + str(vr(ml)) + '})'
